# Basic Settings

## API Keys

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("API_KEY")
os.environ["OPENAI_API_KEY"] = api_key

## Define Tools

In [2]:
from tools import Tools

tools = Tools()
tool_dict = tools.tool_dict

print("This app is using the following tool:")
for tool in tool_dict:
    print(tool)

This app is using the following tool:
none
website_info_retriever
website_links_crawler
website_reader
pdf_reader
create_browser
screen_shot
navigate
get_html_content
input_text_with_label
input_text_with_name
click_button_with_text
click_input_with_label
click_input_with_value
click_input_with_id
select_dropdown_option
click_span_with_aria_label
upload_file_with_id
read_user_input_and_plan
read_execution_chat_log
read_execution_team_agents_prompt
read_evaluation_result


## Read Agent Parameter (yaml)

In [3]:
import yaml

# ! 注意yaml檔案版本
with open('agents_parameter.yaml', 'r', encoding="utf-8") as file:
    agents_parameter = yaml.safe_load(file)

# Execution Team

## Define Agents

### Planner

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import List


class Plan(BaseModel):
    """Plan to follow in future"""

    steps: List[str] = Field(
        description="different steps to follow, should be in sorted order"
    )

planner_llm_config = agents_parameter["Planner"]["llm_config"]
planner_system_prompt = agents_parameter["Planner"]["prompt"]

planner_llm = ChatOpenAI(model=planner_llm_config["model"], temperature=planner_llm_config["temperature"])
planner_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", planner_system_prompt),
        ("placeholder", "{user_input}"), # placeholer 用來動態嵌入使用者輸入的訊息
    ]
)

planner = planner_prompt | planner_llm.with_structured_output(Plan) # 限制使用特定模板回答問題

print("planner_llm_config:")
for key, value in planner_llm_config.items():
    print(f"{key}: {value}")
print("planner_system_prompt: \n" + planner_system_prompt)

planner_llm_config:
model: gpt-4o
temperature: 0
planner_system_prompt: 
You are a Planner Agent in an LLM-based multi-agent system designed to make plans for Executor Agents to follow in order to fulfill user requests by gathering information or operating systems related to National Central University.

Your job is to generate clear, logical, and actionable step-by-step plans that guide other agents to fulfill the user's request. Each plan step should include:
  - A brief explanation of what the step aims to accomplish
  - A clear description of what needs to be found or processed
  - An output placeholder (e.g., #E1, #E2, etc.) for use in later steps

You must use variables like #E1, #E2, etc., to represent intermediate results that can be referenced in later steps. Make sure each step builds clearly upon previous steps. The final step should return the content that most accurately and completely fulfills the user's request.

Please use the following reasoning framework **as a flexib

In [5]:
# response = planner.invoke({"user_input": [("user", "Summarize the content of the 111 Academic Affairs Regulations.")]})
# for step in response.steps:
#     print(step)

In [6]:
# response = planner.invoke({"user_input": [("user", "Please help me fill out the leave application on the school website.")]})
# for step in response.steps:
#     print(step)

### Executor

In [7]:
from agents import create_react_agent_with_yaml

executor = create_react_agent_with_yaml("Executor")

Executor_llm_config:
model: gpt-4o-mini
temperature: 0
Executor_prompt: 
You are an Executor Agent in a multi-agent system designed to assist users in finding and understanding information from school websites.
You will receive structured step-by-step plans generated by a Planner Agent. Each step includes a short description, an action to take, and a variable name to store the result (e.g., #E1 = ...). Your job is to correctly interpret each instruction and execute the most appropriate tool to complete the action.

Tools available to you:
  - website_info_retriever: Retrieves metadata or structured information about a given school website from a pre-built database.
  - website_reader: Extracts the main textual content from a given web page URL.
  - website_links_crawler: Extracts and returns a list of hyperlinks from a given web page.
  - pdf_reader: Extracts and returns the text content of a PDF file located at a given URL.

Execution Rules:
  1. Carefully analyze each task instructio

In [8]:
# response = executor.invoke({"messages": [("user", "Who is the headmaster of National Central University in Taiwan?")]})
# for message in response["messages"]:
#     print(message)

In [9]:
# response = executor.invoke({"messages": [("user", "Please help me fill out the leave application on https://cis.ncu.edu.tw/iNCU/stdAffair/leaveRequest.")]})
# for message in response["messages"]:
#     print(message)

### Replanner

In [10]:
from typing import Union
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI


class Response(BaseModel):
    """Response to user."""

    response: str


class Act(BaseModel):
    """Action to perform."""

    action: Union[Response, Plan] = Field(
        description="Action to perform. If you want to respond to user, use Response."
        "If you need to further use tools to get the answer, use Plan."
    )

replanner_model = agents_parameter["Replanner"]["model"]
replanner_system_prompt = f"{agents_parameter['Replanner']['prompt']}"

replanner_llm = ChatOpenAI(model=replanner_model) # ! Replanner需要使用gpt-4o才不會一直call tools
replanner_prompt = ChatPromptTemplate.from_template(replanner_system_prompt)

replanner = replanner_prompt | replanner_llm.with_structured_output(Act) # 限制使用特定模板回答問題

print("replanner_model: " + replanner_model)
print("replanner_prompt: \n" + replanner_system_prompt)

replanner_model: gpt-4o
replanner_prompt: 
You are a Replanner Agent in a multi-agent system designed to assist users in finding and understanding information from school websites.
For the given objective, come up with a simple step by step plan. \
This plan should involve individual tasks, that if executed correctly will yield the correct answer. Do not add any superfluous steps. \
The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps.

Your objective was this:
{input}

Your original plan was this:
{plan}

You have currently done the following steps:
{past_steps}

If the original website or page appears insufficient (e.g., missing relevant info after one or two visits), consider redirecting the search to more general, authoritative sources—like the university's official homepage—or directly scanning for keywords in navigation items (e.g., "President", "About", "Administration", "Leadership").

Update your p

### Solver

In [11]:
solver_model = agents_parameter["Solver"]["model"]
solver_system_prompt = agents_parameter["Solver"]["prompt"]

solver_llm = ChatOpenAI(model=solver_model)
solver_prompt = ChatPromptTemplate.from_template(solver_system_prompt)

solver = solver_prompt | solver_llm

print("solver_model: " + solver_model)
print("solver_system_prompt: \n" + solver_system_prompt)

solver_model: gpt-4o-mini
solver_system_prompt: 
You are a Solver Agent in a multi-agent system that helps users find and understand information from school websites.

You will receive:
  - The original user request
  - A list of past steps that have been completed
  - The latest available information retrieved by other agents

Your task is to:
  1. Review all available information.
  2. Determine whether the current information is sufficient to answer the user's request.
  3. If it is sufficient, generate a clear and helpful response that directly addresses the user's request.
  4. If it is not sufficient, explain what information is still missing and suggest what to do next.

Be concise, accurate, and helpful. Your response will be shown directly to the user, so make sure it is complete and easy to understand.

Inputs:
  - User request: {user_input}
  - Planning History: {planning_history}

Based on the above, please generate the best possible response to fulfill the user's need.



## Define Graph State

In [12]:
import operator
from typing import Annotated, List, Tuple, Any
from typing_extensions import TypedDict


class PlanExecute(TypedDict):
    input: str
    plan: List[str]
    past_steps: Annotated[List[Tuple], operator.add]
    response: str
    history: List[Tuple[str, Any]]

## Define Agent Node

In [13]:
async def plan_step(state: PlanExecute):
    plan = await planner.ainvoke({"user_input": [("user", state["input"])]}) # 對應到planner system prompt中的{user_input}
    state["history"].append(("Planner", plan.steps)) # 將plan的步驟加入history中

    return {
        "plan": plan.steps,
        "history": state["history"],
    }

async def execute_step(state: PlanExecute):
    plan = state["plan"]
    plan_str = "\n".join(f"{i+1}. {step}" for i, step in enumerate(plan))
    task = plan[0]
    task_formatted = f"""For the following plan:
{plan_str}\n\nYou are tasked with executing step {1}, {task}."""
    agent_response = await executor.ainvoke({"messages": [("user", task_formatted)]}) # react agent 用 messages 方式接收訊息
    state["history"].append(("Executor", (task, agent_response["messages"][-1].content)))

    return {
        "past_steps": [(task, agent_response["messages"][-1].content)], # react agent 接收訊息方式
        "history": state["history"],
    }

async def replan_step(state: PlanExecute):
    # 過濾掉state中不需要的欄位
    temp_state = state.copy()
    temp_state.pop("history")

    output = await replanner.ainvoke(temp_state)
    if isinstance(output.action, Response):
        state["history"].append(("Replanner", output.action.response))
        return {
            "response": output.action.response,
            "history": state["history"],
        }
    else:
        state["history"].append(("Replanner", output.action.steps))
        return {
            "plan": output.action.steps,
            "history": state["history"],
        }

async def solve_step(state: PlanExecute):
    print("history:")
    print(state["history"])
    response = await solver.ainvoke({"user_input": state["input"], "planning_history": state["history"]})
    return {"response": response.content, "history": state["history"]}

def should_end(state: PlanExecute):
    if "response" in state and state["response"]:
        return "solver"
    else:
        return "executor"

## Create Graph

In [14]:
from langgraph.graph import StateGraph, START, END

workflow = StateGraph(PlanExecute)

workflow.add_node("planner", plan_step)
workflow.add_node("executor", execute_step)
workflow.add_node("replanner", replan_step)
workflow.add_node("solver", solve_step)

workflow.add_edge(START, "planner")
workflow.add_edge("planner", "executor")
workflow.add_edge("executor", "replanner")
workflow.add_conditional_edges(
    "replanner",
    # Next, we pass in the function that will determine which node is called next.
    should_end,
    ["executor", "solver"],
)
workflow.add_edge("solver", END)

app = workflow.compile() # This compiles it into a LangChain Runnable, meaning you can use it as you would any other runnable

In [15]:
# from IPython.display import Image, display

# display(Image(app.get_graph(xray=True).draw_mermaid_png()))

## Run App

In [16]:
sequence = 0

with open("Outputs/execution_chat_log.txt", "w") as f:
    f.write("")

def write_to_chat_log(content):
    with open("Outputs/execution_chat_log.txt", "a") as f:
        f.write(content)

# Who is the headmaster of National Central University in Taiwan?
# Summarize the content of the 111 Academic Affairs Regulations.
# Please help me fill out the leave application on the school website.
config = {"recursion_limit": 30}
inputs = {
    "input": "Who is the headmaster of National Central University in Taiwan?",
    "history": [], # 初始化儲存History的list
}
write_to_chat_log(f"User Query:\n{inputs['input']}\n\n")

# tool_dict["create_browser"].invoke(input=None)

async for event in app.astream(inputs, config=config):
    for agent, state in event.items():
        if agent != "__end__":
            write_to_chat_log(f"{agent}:\n")
            # ! Jupyter Notebook 裡使用 global sequence 會報錯，需要使用 nest_asyncio
            # global sequence
            # sequence += 1
            # write_to_chat_log(f"{sequence}. {agent}:\n")

            for key, value in state.items():
                if (key != "history"):
                    write_to_chat_log(f"{key}: {value}\n")
            
            write_to_chat_log("\n")

# del tools.selenium_controller

Query: 國立中央大學 官方網站
---
link:  https://www.ncu.edu.tw/tw/index.php/#B
page_content: 
 網站標題: :::
簡介: 國立中央大學的官網提供了多方面的信息和服務，包括校園活動、教學單位、研究中心、行政服務等。網站上有最新的新聞專區，報導校內活動，如舞蹈大賽、獲獎資訊及學術研究進展。此外，網站也提供了招生資訊、校園紀念品、捐贈途徑及各類講座和工作坊的公告。校園活動的日曆顯示了運動會等重要活動的安排。網站支持多種身份入口，方便教職員、學生、校友及家長訪問相關資訊。網站使用Cookie以提升使用體驗，並建議使用現代瀏覽器以獲得最佳效果。
---
link:  https://pdc.adm.ncu.edu.tw/.//#U
page_content: 
 網站標題: :::
簡介: 國立中央大學的教務處網站提供了多項功能與資訊，包括招生、註冊、課務以及教學發展的相關內容。網站上有最新消息的公告，涵蓋了研究所甄試、課程停修申請、交換學生申請等重要通知。此外，教學發展中心也舉辦了各類工作坊和競賽，鼓勵學生參與跨領域學習。網站建議使用最新版本的瀏覽器以獲得最佳瀏覽效果，並提供聯絡資訊及隱私政策聲明。
---
link:  https://www.ncu.edu.tw/tw/index.php/#U
page_content: 
 網站標題: :::
簡介: 國立中央大學的官方網站提供了多元化的資訊與服務，涵蓋教學單位、研究中心、行政服務及校內活動等內容。網站強調用戶友好性，並提供了多種快捷鍵和功能說明，以協助使用者有效瀏覽。近期焦點新聞包括校內運動會、獲得友善校園獎的校務長及與國際大學的合作訪問等活動。

網站還有專區針對招生、研究發展、校園紀念品及捐贈活動進行介紹。此外，校內活動日曆列出了即將舉行的各類活動，包括運動會及講座等。聯絡資訊、重要連結及隱私政策等也在網站上有所提供，確保使用者能獲得全面的資訊和支持。
---
link:  https://pdc.adm.ncu.edu.tw/#U/#C
page_content: 
 網站標題: :::
簡介: 國立中央大學的教務處網站提供了多項服務及最新消息，包括招生資訊、課程管理、教學發展等功能。網站設有各個組別如招生組、註冊組、課務組和教發中心，

c:\Users\George\anaconda3\envs\langgraph_03_28_25\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ncu.edu.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\George\anaconda3\envs\langgraph_03_28_25\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ncu.edu.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\George\anaconda3\envs\langgraph_03_28_25\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ncu.edu.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  w

無法獲取[網站導覽]: [https://www.ncu.edu.tw/tw/pages/sitemap.php] ，不加入資料中。錯誤: HTTPSConnectionPool(host='www.ncu.edu.tw', port=443): Read timed out. (read timeout=1)


c:\Users\George\anaconda3\envs\langgraph_03_28_25\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ncu.edu.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\George\anaconda3\envs\langgraph_03_28_25\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'portal.ncu.edu.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\George\anaconda3\envs\langgraph_03_28_25\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'portal.ncu.edu.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnin

成功獲取 [https://www.ncu.edu.tw/tw/index.php] 中共135個連結。
[跳到主要內容]: [https://www.ncu.edu.tw/tw/index.php/#C]
[:::]: [https://www.ncu.edu.tw/tw/index.php/#U]
[網站導覽]: [https://www.ncu.edu.tw/tw/pages/sitemap.php]
[教職員]: [https://www.ncu.edu.tw/tw/pages/index.php?num=245]
[學生]: [https://www.ncu.edu.tw/tw/pages/index.php?num=246]
[校友]: [https://www.ncu.edu.tw/tw/pages/index.php?num=247]
[家長]: [https://www.ncu.edu.tw/tw/pages/index.php?num=248]
[訪客]: [https://www.ncu.edu.tw/tw/pages/index.php?num=249]
[中大Portal]: [https://portal.ncu.edu.tw/login]
[關於中大]: [https://www.ncu.edu.tw/tw/pages/index.php?num=1]
[教學單位]: [https://www.ncu.edu.tw/tw/pages/index.php?num=2]
[研究中心]: [https://www.ncu.edu.tw/tw/pages/index.php?num=489]
[行政服務]: [https://www.ncu.edu.tw/tw/pages/index.php?num=3]
[全站搜尋鈕]: [https://www.ncu.edu.tw/tw/index.php/javascript:$('#ncuForm').submit();]
[行政服務]: [https://www.ncu.edu.tw/tw/pages/Advanced-Search.php?q=行政服務]
[轉學辦理]: [https://www.ncu.edu.tw/tw/pages/Advanced-Search.php?q=轉學辦理]
[開學

...

成功讀取 [https://www.ncu.edu.tw/tw/pages/index.php?num=3] 並總結。
國立中央大學 - 行政服務
  您的瀏覽器不支援JavaScript語法，JavaScript語法並不影響內容的陳述。您可使用按鍵盤上的Ctrl鍵+ (+)鍵放大/(-)鍵縮小來改變字型大小；回到上一頁可使用瀏覽器提供的 Alt+左方向鍵(←) 快速鍵功能；列印可使用瀏覽器提供的(Ctrl+P)功能。
歡迎光臨 國立中央大學
跳到主要內容
:::
網站導覽
教職員
學生
校友
家長
訪客
中大Portal
English
溫度：26ﾟC
濕度：32%
關於中大
教學單位
研究中心
行政服務
全站搜尋
全站搜尋鈕
熱門搜尋：
                                    行政服務
轉學辦理
開學
手機版側邊欄選單
手機版搜尋功能
全站搜尋
全文檢索
熱門搜尋：
                        行政服務
轉學辦理
開學
關於中大
教學單位
研究中心
行政服務
身份入口
教職員
學生
校友
家長
訪客
重要連結
中大校曆 
校園地圖 
自辦品保專區 
中大Portal 
國際產學聯盟 
台灣聯合大學系統 
學研人才庫 
大學社會責任 
性別平等教育 
延攬優秀人才返國服務 
身心障礙甄試委員會 
其他連結 
網站導覽
回首頁
中大Portal
English
分享至
:::
 首頁
行政服務
字級：
小 
中 
大
列印
社群分享
行政服務
校長/副校長
 蕭述三校長
 綦振瀛副校長
 陳文逸副校長
 阮啓弘副校長
教務處
 註冊組
 課務組
 招生組
 教學發展中心
 中大出版中心
學生事務處
 生活輔導組
 課外活動組
 衛生保健組
 住宿服務組
 職涯發展中心
 諮商輔導中心
 服務學習發展中心
 原住民族學生資源中心
總務處
 文書組
 事務組
 出納組
 營繕組
 資產經營管理組
 採購組
 校務人力組
 駐校警衛隊
研究發展處
 企劃組
 學術發展組
 計畫管理組
 產學營運中心
 貴重儀器中心
國際事務處
 國際行政事務組
 國際交流暨招生組
 海外教育推廣組
秘書室
 秘書室
 第一組
 第二組
 校友服務中心
圖書館
 採編組
 典閱組
 推廣服務組
 資訊系統組
電子計算機中心
 技術研發組


c:\Users\George\anaconda3\envs\langgraph_03_28_25\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ncu.edu.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


成功讀取 [https://www.ncu.edu.tw] 並總結。
國立中央大學
國立中央大學


c:\Users\George\anaconda3\envs\langgraph_03_28_25\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ncu.edu.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


成功讀取 [https://www.ncu.edu.tw/en/index.php] 並總結。
National Central University
  Your browser does not support JavaScript,but it does not matter.JavaScript syntax here does not affect the content of the statement.If you need to select your text size,please press Ctrl + (+) enlarge(-) reduce to change the font size. Sitemap
welcome to National Central University
Skip to main content block
:::
Sitemap
Staff & Faculty
Students
Alumni
Parents
Visitor
NCU Portal
中文版
Temp：26ﾟC
RH：32%
About NCU
Colleges and Centers
Research Centers
Administration
Sitewide Search
Full Text Search Button
Popular Searches:
                                    5G
Exchange Students
covid-19
mobile menu
Mobile Sitewide Search
Sitewide Search
Full Text Search
Popular Searches：
                    5G
Exchange Students
covid-19
About NCU
Colleges and Centers
Research Centers
Administration
Identity
Staff & Faculty
Students
Alumni
Parents
Visitor
Important Link
Calendar 
Campus Map 
Recognition of Self Accreditation 
NCU P

# Evaluation Team

## Define Agents

### Critic

In [ ]:
from agents import create_react_agent_with_yaml

# * 根據使用者輸入和計畫制定生成評估標準
critic = create_react_agent_with_yaml("Critic")

In [ ]:
# response = critic.invoke({"messages": [("user", "Please evaluate the performance of execution team.")]})

# # 暫存評估標準，之後儲存到state內交給evaluator
# with open("Docs/evaluation_rubric.txt", "w") as f:
#     f.write(f"{response['messages'][-1].content}\n\n")

In [ ]:
# print(response["messages"][-1].content)

In [ ]:
# 查看調用工具情形
# for message in response["messages"]:
#     print(message)
#     if not message.content:
#         for item in message:
#             print(item)

### Evaluator

In [ ]:
from agents import create_react_agent_with_yaml

# * 根據評估者提供的評估框架和評估執行團隊的任務執行成效
evaluator = create_react_agent_with_yaml("Evaluator")

In [ ]:
# with open('Docs/evaluation_rubric.txt', 'r') as file:
#     evaluation_rubric = file.read()

# response = evaluator.invoke({"messages": [("user", evaluation_rubric)]})

# # 暫存評估結果，之後儲存到state內交給analyzer
# with open("evaluation_result.txt", "w") as f:
#     f.write(f"{response['messages'][-1].content}\n\n")

In [ ]:
# print(response["messages"][-1].content)

In [ ]:
# # 查看調用工具情形
# for message in response["messages"]:
#     print(message)
#     if not message.content:
#         for item in message:
#             print(item)

### Analyzer

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

analyzer_llm_config = agents_parameter["Analyzer"]["llm_config"]
analyzer_system_prompt = agents_parameter["Analyzer"]["prompt"]

analyzer_llm = ChatOpenAI(model=analyzer_llm_config["model"], temperature=analyzer_llm_config["temperature"])
analyzer_prompt = ChatPromptTemplate.from_template(analyzer_system_prompt)

analyzer = analyzer_prompt | analyzer_llm

print("analyzer_llm_config:")
for key, value in analyzer_llm_config.items():
    print(f"{key}: {value}")
print("analyzer_system_prompt: \n" + analyzer_system_prompt)

In [ ]:
# with open("Docs/evaluation_result.txt") as file:
#     evaluation_result = file.read()

# response = analyzer.invoke({"evaluation_result": evaluation_result})

In [ ]:
# print(response.content)

## Define Graph State

In [ ]:
from typing_extensions import TypedDict

class Evaluation(TypedDict):
    input: str
    rubric: str
    result: str
    judgment: str

## Define Agent Node

In [ ]:
async def critic_step(state: Evaluation):
    response = await critic.ainvoke({"messages": [("user", state["input"])]})
    state["rubric"] = response["messages"][-1].content # 儲存評估標準到state內
    return {
        "rubric": state["rubric"],
    }

async def evaluator_step(state: Evaluation):
    response = await evaluator.ainvoke({"messages": [("user", state["rubric"])]})
    state["result"] = response["messages"][-1].content # 儲存評估結果到state內
    return {
        "result": state["result"],
    }

async def analyzer_step(state: Evaluation):
    response = await analyzer.ainvoke({"evaluation_result": state["result"]})
    state["judgment"] = response.content # 儲存分析結果到state內
    return {
        "judgment": state["judgment"],
    }

## Create Graph

In [ ]:
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display

evaluation_workflow = StateGraph(Evaluation)

evaluation_workflow.add_node("critic", critic_step)
evaluation_workflow.add_node("evaluator", evaluator_step)
evaluation_workflow.add_node("analyzer", analyzer_step)

evaluation_workflow.add_edge(START, "critic")
evaluation_workflow.add_edge("critic", "evaluator")
evaluation_workflow.add_edge("evaluator", END)
# evaluation_workflow.add_edge("evaluator", "analyzer")
# evaluation_workflow.add_edge("analyzer", END)

evaluation_app = evaluation_workflow.compile() # This compiles it into a LangChain Runnable, meaning you can use it as you would any other runnable

display(Image(evaluation_app.get_graph(xray=True).draw_mermaid_png()))

## Run App

In [ ]:
sequence = 0

with open("evaluation_chat_log.txt", "w") as f:
    f.write("")

def write_to_chat_log(content):
    with open("evaluation_chat_log.txt", "a") as f:
        f.write(content)

# Please evaluate the performance of execution team.
config = {"recursion_limit": 50}
inputs = {
    "input": "Please evaluate the performance of execution team.",
}
write_to_chat_log(f"Evaluation Query:\n{inputs['input']}\n\n")

async for event in evaluation_app.astream(inputs, config=config):
    for agent, state in event.items():
        if agent != "__end__":
            write_to_chat_log(f"{agent}:\n")
            # ! Jupyter Notebook 裡使用 global sequence 會報錯，需要使用 nest_asyncio
            # global sequence
            # sequence += 1
            # write_to_chat_log(f"{sequence}. {agent}:\n")

            for key, value in state.items():
                if (key != "history"):
                    write_to_chat_log(f"{key}: {value}\n")
            
            write_to_chat_log("\n")

# Evolution Team

## Define Agents

### Analyzer

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

analyzer_llm_config = agents_parameter["Analyzer"]["llm_config"]
analyzer_system_prompt = agents_parameter["Analyzer"]["prompt"]

analyzer_llm = ChatOpenAI(model=analyzer_llm_config["model"], temperature=analyzer_llm_config["temperature"])
analyzer_prompt = ChatPromptTemplate.from_template(analyzer_system_prompt)

analyzer = analyzer_prompt | analyzer_llm

print("analyzer_llm_config:")
for key, value in analyzer_llm_config.items():
    print(f"{key}: {value}")
print("analyzer_system_prompt: \n" + analyzer_system_prompt)

### Prompt Optimizer